https://www.youtube.com/watch?v=POjDCe-_G8k&list=PLBJe2dFI4sgukOW6O0B-OVyX9c6fQKJ2N&index=3

https://www.kaggle.com/datasets/csanhueza/the-marvel-universe-social-network

1: Create a table with PostgreSQL

In [1]:
import psycopg2
import pandas as pd

extract data from .csv files

In [2]:
edgesdf = pd.read_csv("edges.csv")
herodf = pd.read_csv("hero-network.csv")
nodesdf = pd.read_csv("nodes.csv")

In [3]:
edgesdf.head()

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


In [21]:
edgesdf.describe()

,hero,comic
count,96104,96104
unique,6439,12651
top,SPIDER-MAN/PETER PARKER,COC 1
freq,1577,111


In [4]:
herodf.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


In [5]:
nodesdf.head()

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero


create connection to database

In [6]:
def create_db():
    conn = psycopg2.connect("host = localhost dbname = postgres user=postgres password = Lakshmi@1")
    conn.set_session(autocommit = True)
    cur = conn.cursor()
    
    cur.execute("DROP DATABASE marvel_heros")
    cur.execute("CREATE DATABASE marvel_heros")
    
    cur.close()
    conn.close()
    
    conn = psycopg2.connect("host = localhost dbname = marvel_heros user=postgres password = Lakshmi@1")
    cur = conn.cursor() 
    
    return cur, conn

In [7]:
def drop_tables(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [8]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

Create connection and cursor 

In [9]:
cur, conn = create_db()

Create edges, heros, nodes tables in marvel_heros database

In [10]:
edges_table = ("CREATE TABLE IF NOT EXISTS edges(hero VARCHAR, comic varchar)")

In [11]:
cur.execute(edges_table)
conn.commit()

In [12]:
heros_table = ("CREATE TABLE IF NOT EXISTS heros(hero1 VARCHAR, hero2 varchar)")

In [13]:
cur.execute(heros_table)
conn.commit()

In [14]:
nodes_table = ("CREATE TABLE IF NOT EXISTS nodes(node VARCHAR, type varchar)")

In [15]:
cur.execute(nodes_table)
conn.commit()

In [16]:
cur.execute("ROLLBACK")
conn.commit()

Insert values from dataframes to data tables

In [17]:
edges_insert = ("INSERT INTO edges(hero, comic) VALUES (%s, %s)")

In [22]:
for i, row in edgesdf.iterrows():
    cur.execute(edges_insert, row)

In [23]:
heros_insert = ("INSERT INTO heros(hero1, hero2) VALUES (%s, %s)")
for i, row in herodf.iterrows():
    cur.execute(heros_insert, list(row))

In [24]:
nodes_insert = ("INSERT INTO nodes(node, type) VALUES (%s, %s)")

In [25]:
for i, row in nodesdf.iterrows():
    cur.execute(nodes_insert, list(row))

In [26]:
conn.commit()

Finally close cursor and connection

In [27]:
cur.close()
conn.close()